In [1]:
import pandas as pd
import numpy as np

In [2]:
from constants import cleaned_data_filepath
participants = ["107", "904", "905", "906"] #, "111", "908"]

from ml_prep import prep_ml
downsample_num = 100
X,y,p,w = prep_ml(cleaned_data_filepath, participants, averaging="average_trials_and_participants", downsample_num=downsample_num)

/Users/JennMacBook/Desktop/Studies/Animates EEG/8_Github/jwlab_eeg/classification/ml_prep.py:98: RuntimeWarning: Mean of empty slice.
  means = df_data[df.label == w].values.mean()
/Users/JennMacBook/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: too many values to unpack (expected 4)

In [ ]:
y.shape[0]

In [ ]:
(n, d) = X.shape
assert n == y.shape[0]

In [ ]:
from sklearn.svm import LinearSVC
model = LinearSVC(C=1e-6)

In [ ]:
# from sklearn.ensemble import BaggingClassifier
# model = BaggingClassifier(model)

In [ ]:
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

def dist(a, b):
    a = np.reshape(a, (-1, downsample_num))
    b = np.reshape(b, (-1, downsample_num))
    
    dists = np.array([fastdtw(a[i, :], b[i, :])[0] for i in range(a.shape[0])])
    print(dists)
    
    return np.sum(np.square(dists))
    

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV


parameters = {'n_neighbors':[2, 4, 8]}
# model = GridSearchCV(KNeighborsClassifier(metric=DTW), parameters, cv=3, verbose=1)
model = KNeighborsClassifier(metric=dist, n_neighbors=1)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size=0.2, random_state=20)
model.fit(X_train, y_train)

In [ ]:
# from sklearn.model_selection import GridSearchCV

# param_grid = [
  # {'C': np.logspace(-10, -5, 3)}
 # ]

# tuned_model = GridSearchCV(model, param_grid)
# tuned_model.fit(X, y)

# tuned_model.cv_results_

In [ ]:
np.mean(model.predict(X_test) != y_test)

In [ ]:
from eval import eval_normal
errs = eval_normal(model, X, y, 100, random_state=21)
np.mean(errs)

In [ ]:
from eval import eval_across_categories
errs = eval_across_categories(model, X, y, p, 100, random_state=43598)
print("\n---\n")
for j in range(errs.shape[0]):
   print(np.mean(errs[j, :]))

In [ ]:
errs2 = eval_across_categories(model, X, y, w, 100, random_state=498)
print("\n---\n")
for j in range(errs2.shape[0]):
   print(np.mean(errs2[j, :]))